In [1]:
import os
import cv2
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm

import timm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data.sampler import Sampler
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau

import albumentations as A
from albumentations.pytorch import ToTensor

from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score

import warnings
warnings.filterwarnings(action='ignore')

N_EPOCHS = 1000
BATCH_SIZE = 128
LEARNING_RATE = 0.0005
PAITIENCE = 30

IM_HEIGHT = 256
IM_WIDTH = 256

In [2]:
train_pos, train_neg  = 'data/tumor_classification/train/positive', 'data/tumor_classification/train/negative'
valid_pos, valid_neg  = 'data/tumor_classification/valid/positive', 'data/tumor_classification/valid/negative'
test_pos, test_neg    = 'data/tumor_classification/test/positive', 'data/tumor_classification/test/negative'

def generate_dataframe(path_positive, path_negative):
    df = pd.concat([
            pd.DataFrame({"image": glob.glob(path_positive + "/*.png"), "target": 1}),
            pd.DataFrame({"image": glob.glob(path_negative + "/*.png"), "target": 0}),
        ])
    
    return df

train_df = generate_dataframe(train_pos, train_neg)
valid_df = generate_dataframe(valid_pos, valid_neg)
test_df  = generate_dataframe(test_pos, test_neg)

In [3]:
train_transforms = A.Compose([
    A.Resize(width=IM_HEIGHT, height=IM_WIDTH, p=1.0),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.ShiftScaleRotate(p=0.5, shift_limit=0.05, scale_limit=0.1, rotate_limit=45),
    A.ElasticTransform(p=0.5),
    A.HueSaturationValue(p=0.5),
    A.RandomBrightness(p=0.5),
    A.Transpose(p=0.5),
    A.GaussNoise(p=0.3),
    A.Blur(p=0.3),
    A.JpegCompression(p=0.3),
    A.Normalize(p=1.0),
    ToTensor(),
])

valid_transforms = A.Compose([
    A.Resize(width=IM_HEIGHT, height=IM_WIDTH, p=1.0),
    A.Normalize(p=1.0),
    ToTensor(),
])


class TumorDataset(Dataset):
    def __init__(self, df, transforms):
        self.df = df
        self.transforms = transforms
        
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, idx):
        image  = cv2.imread(self.df.iloc[idx, 0])
        target = self.df.iloc[idx, 1]

        augmented = self.transforms(image=image)
        image = augmented['image']  
        
        return image, target
    
    def get_labels(self):
        return list(self.df.target.values)

    
class BalanceClassSampler(Sampler):
    """Abstraction over data sampler.
    Allows you to create stratified sample on unbalanced classes.
    """

    def __init__(self, labels, mode="downsampling"):
        """
        Args:
            labels (List[int]): list of class label
                for each elem in the datasety
            mode (str): Strategy to balance classes.
                Must be one of [downsampling, upsampling]
        """
        super().__init__(labels)

        labels = np.array(labels)
        samples_per_class = {
            label: (labels == label).sum() for label in set(labels)
        }

        self.lbl2idx = {
            label: np.arange(len(labels))[labels == label].tolist()
            for label in set(labels)
        }

        if isinstance(mode, str):
            assert mode in ["downsampling", "upsampling"]

        if isinstance(mode, int) or mode == "upsampling":
            samples_per_class = (
                mode
                if isinstance(mode, int)
                else max(samples_per_class.values())
            )
        else:
            samples_per_class = min(samples_per_class.values())

        self.labels = labels
        self.samples_per_class = samples_per_class
        self.length = self.samples_per_class * len(set(labels))

    def __iter__(self):
        """
        Yields:
            indices of stratified sample
        """
        indices = []
        for key in sorted(self.lbl2idx):
            replace_ = self.samples_per_class > len(self.lbl2idx[key])
            indices += np.random.choice(
                self.lbl2idx[key], self.samples_per_class, replace=replace_
            ).tolist()
        assert len(indices) == self.length
        np.random.shuffle(indices)

        return iter(indices)

    def __len__(self) -> int:
        """
        Returns:
             length of result sample
        """
        return self.length
    
    
train_dataset = TumorDataset(df=train_df, transforms=train_transforms)
# train_iterator = DataLoader(train_dataset, batch_size=BATCH_SIZE, num_workers=4, shuffle=True)
train_iterator = DataLoader(train_dataset, batch_size=BATCH_SIZE, num_workers=4, shuffle=False, 
    sampler=BalanceClassSampler(labels=train_dataset.get_labels(), mode='downsampling'))

valid_dataset = TumorDataset(df=valid_df, transforms=valid_transforms)
# valid_iterator = DataLoader(valid_dataset, batch_size=BATCH_SIZE, num_workers=4, shuffle=True)
valid_iterator = DataLoader(valid_dataset, batch_size=BATCH_SIZE, num_workers=4, shuffle=False,
    sampler=BalanceClassSampler(labels=valid_dataset.get_labels(), mode='downsampling'))

test_dataset = TumorDataset(df=test_df, transforms=valid_transforms)
test_iterator = DataLoader(test_dataset, batch_size=1, num_workers=4, shuffle=False)

In [4]:
device       = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model        = timm.create_model('resnet50', num_classes=2, pretrained=True).to(device)
optimizer    = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler    = ReduceLROnPlateau(optimizer, 'min')
class_weight = torch.tensor([1.1, 1.0]).to(device)
criterion    = nn.CrossEntropyLoss(weight=class_weight)

for name, param in model.named_parameters():
    param.requires_grad = True
#     if name[:4] == "head":
#         param.requires_grad = True
#     else:
#         param.requires_grad = False

In [5]:
def train(model, iterator, criterion, optimizer, device=device):  
    model.train()
    epoch_loss = 0
    correct    = 0    
    
    for image, target in iterator:
        optimizer.zero_grad()
        image  = image.to(device)
        target = target.long().to(device)
        
        output = model(image).squeeze()
        
        loss = criterion(output, target)
        loss.backward()
        epoch_loss += loss.item()
        
        pred     = torch.argmax(output, axis=1)
        correct += (pred == target).sum()

        optimizer.step()

    return epoch_loss / len(iterator.dataset), correct / len(iterator.dataset)


@torch.no_grad()
def evaluate(model, iterator, criterion, device=device):
    model.eval()
    epoch_loss = 0
    correct    = 0 
    
    for image, target in iterator:
        image  = image.to(device)
        target = target.long().to(device)

        output = model(image).squeeze()

        loss = criterion(output, target)
        epoch_loss += loss.item()

        pred     = torch.argmax(output, axis=1)
        correct += (pred == target).sum()
            
    return epoch_loss / len(iterator.dataset), correct / len(iterator.dataset)


@torch.no_grad()
def predict(model, iterator, device=device):
    model.eval()
    pred = []
    true = []
    
    for image, target in iterator:
        image  = image.to(device)
        target = target.long().to(device)

        output = model(image)

        pred.append(output.to("cpu").tolist()[0])
        true.append(target.to("cpu").tolist()[0])

    return np.argmax(pred, axis=1), true


def print_train_log(epoch_num, train_loss, valid_loss, train_acc, valid_acc):
    print(f"EPOCH: {epoch_num:04}")
    print(f"Train loss: {round(train_loss, 4)}\tTrain acc : {round(float(train_acc), 4)}\tValid loss: {round(valid_loss, 4)}\tValid acc : {round(float(valid_acc), 4)}")
    
    
def compute_test_metrics(true, pred):
    confusion_mat = confusion_matrix(true, pred)
    accuracy      = accuracy_score(true, pred)
    precision     = precision_score(true, pred)
    recall        = recall_score(true, pred)
    f1            = f1_score(true, pred)
    
    return confusion_mat, accuracy, precision, recall, f1


def print_test_log(epoch_num, accuracy, precision, recall, f1):
    print(f"EPOCH: {epoch_num:04} prediction results ")
    print(f"confusion matrix\n{confusion_mat}")
    print(f"accuracy score  : {round(accuracy, 4)}")
    print(f"precision score : {round(precision, 4)}")
    print(f"recall score    : {round(recall, 4)}")
    print(f"f1 score        : {round(f1, 4)}")

In [6]:
start_epoch = 0
if len(glob.glob("output/tumor_classifiaction/resnet50/*.txt")) != 0:
    print("load trained model ... ")
    start_epoch = len(glob.glob("output/tumor_classifiaction/resnet50/*.txt")) - 1 
    model.load_state_dict(torch.load('weights/tumor_classification/best_resnet50.pt'))

n_paitience = 0
best_valid_loss = float('inf')

optimizer.zero_grad()
optimizer.step()

for epoch_num in range(start_epoch, N_EPOCHS):
    train_loss, train_acc = train(model, train_iterator, criterion, optimizer, device)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion, device)
    
    scheduler.step(valid_loss)
    
    print_train_log(epoch_num, train_loss, valid_loss, train_acc, valid_acc)
    with open("output/tumor_classification/resnet50/log.txt", "a") as f:
        f.write("epoch: {0:04d} train loss: {1:.4f}, valid loss: {2:.4f}, train Acc: {3:.4f}, valid Acc: {4:.4f}\n".format(epoch_num, train_loss, valid_loss, train_acc, valid_acc))

    if n_paitience < PAITIENCE:
        if best_valid_loss > valid_loss:
            best_valid_loss = valid_loss
            torch.save(model.state_dict(), 'weights/tumor_classification/best_resnet50.pt')
            n_paitience = 0
        elif best_valid_loss <= valid_loss:
            n_paitience += 1
    else:
        print("Early stop!")
        model.load_state_dict(torch.load('weights/tumor_classification/best_resnet50.pt'))
        break
        
    if epoch_num % 1 == 0:
        
        pred, true = predict(model, test_iterator)
        confusion_mat, accuracy, precision, recall, f1 = compute_test_metrics(true, pred)
        
        print_test_log(epoch_num, accuracy, precision, recall, f1)
        with open("output/tumor_classification/resnet50/epoch_{0:04d}_eval_metrics.txt".format(epoch_num), "a") as f:
            f.write("accuracy score: {0:.4f}, precision score: {1:.4f}, recall score: {2:.4f}, f1 score: {3:.4f}\n".format(accuracy, precision, recall, f1))

EPOCH: 0000
Train loss: 0.0017	Train acc : 0.5242	Valid loss: 0.001	Valid acc : 0.5781
EPOCH: 0000 prediction results 
confusion matrix
[[ 738    7]
 [ 199 1454]]
accuracy score  : 0.9141
precision score : 0.9952
recall score    : 0.8796
f1 score        : 0.9338
EPOCH: 0001
Train loss: 0.0012	Train acc : 0.5576	Valid loss: 0.0008	Valid acc : 0.5834
EPOCH: 0001 prediction results 
confusion matrix
[[ 690   55]
 [  83 1570]]
accuracy score  : 0.9425
precision score : 0.9662
recall score    : 0.9498
f1 score        : 0.9579
EPOCH: 0002
Train loss: 0.0011	Train acc : 0.5625	Valid loss: 0.0006	Valid acc : 0.5939
EPOCH: 0002 prediction results 
confusion matrix
[[ 717   28]
 [  68 1585]]
accuracy score  : 0.96
precision score : 0.9826
recall score    : 0.9589
f1 score        : 0.9706
EPOCH: 0003
Train loss: 0.001	Train acc : 0.573	Valid loss: 0.0006	Valid acc : 0.5912
EPOCH: 0003 prediction results 
confusion matrix
[[ 706   39]
 [  64 1589]]
accuracy score  : 0.957
precision score : 0.976
r

EPOCH: 0034 prediction results 
confusion matrix
[[ 706   39]
 [  27 1626]]
accuracy score  : 0.9725
precision score : 0.9766
recall score    : 0.9837
f1 score        : 0.9801
EPOCH: 0035
Train loss: 0.0006	Train acc : 0.5917	Valid loss: 0.0003	Valid acc : 0.6054
EPOCH: 0035 prediction results 
confusion matrix
[[ 731   14]
 [  41 1612]]
accuracy score  : 0.9771
precision score : 0.9914
recall score    : 0.9752
f1 score        : 0.9832
EPOCH: 0036
Train loss: 0.0006	Train acc : 0.5917	Valid loss: 0.0004	Valid acc : 0.6028
EPOCH: 0036 prediction results 
confusion matrix
[[ 726   19]
 [  43 1610]]
accuracy score  : 0.9741
precision score : 0.9883
recall score    : 0.974
f1 score        : 0.9811
EPOCH: 0037
Train loss: 0.0006	Train acc : 0.5919	Valid loss: 0.0004	Valid acc : 0.6044
EPOCH: 0037 prediction results 
confusion matrix
[[ 724   21]
 [  33 1620]]
accuracy score  : 0.9775
precision score : 0.9872
recall score    : 0.98
f1 score        : 0.9836
EPOCH: 0038
Train loss: 0.0006	Trai

EPOCH: 0066
Train loss: 0.0004	Train acc : 0.6025	Valid loss: 0.0003	Valid acc : 0.6071
EPOCH: 0066 prediction results 
confusion matrix
[[ 734   11]
 [  19 1634]]
accuracy score  : 0.9875
precision score : 0.9933
recall score    : 0.9885
f1 score        : 0.9909
EPOCH: 0067
Train loss: 0.0004	Train acc : 0.6047	Valid loss: 0.0003	Valid acc : 0.6093
EPOCH: 0067 prediction results 
confusion matrix
[[ 734   11]
 [  17 1636]]
accuracy score  : 0.9883
precision score : 0.9933
recall score    : 0.9897
f1 score        : 0.9915
EPOCH: 0068
Train loss: 0.0004	Train acc : 0.604	Valid loss: 0.0003	Valid acc : 0.6099
EPOCH: 0068 prediction results 
confusion matrix
[[ 735   10]
 [  19 1634]]
accuracy score  : 0.9879
precision score : 0.9939
recall score    : 0.9885
f1 score        : 0.9912
EPOCH: 0069
Train loss: 0.0004	Train acc : 0.6042	Valid loss: 0.0003	Valid acc : 0.6091
EPOCH: 0069 prediction results 
confusion matrix
[[ 735   10]
 [  18 1635]]
accuracy score  : 0.9883
precision score : 0.

EPOCH: 0097 prediction results 
confusion matrix
[[ 737    8]
 [  19 1634]]
accuracy score  : 0.9887
precision score : 0.9951
recall score    : 0.9885
f1 score        : 0.9918
EPOCH: 0098
Train loss: 0.0003	Train acc : 0.606	Valid loss: 0.0002	Valid acc : 0.6106
EPOCH: 0098 prediction results 
confusion matrix
[[ 735   10]
 [  14 1639]]
accuracy score  : 0.99
precision score : 0.9939
recall score    : 0.9915
f1 score        : 0.9927
EPOCH: 0099
Train loss: 0.0003	Train acc : 0.6079	Valid loss: 0.0003	Valid acc : 0.6099
EPOCH: 0099 prediction results 
confusion matrix
[[ 736    9]
 [  14 1639]]
accuracy score  : 0.9904
precision score : 0.9945
recall score    : 0.9915
f1 score        : 0.993
EPOCH: 0100
Train loss: 0.0003	Train acc : 0.6088	Valid loss: 0.0003	Valid acc : 0.6089
EPOCH: 0100 prediction results 
confusion matrix
[[ 735   10]
 [  17 1636]]
accuracy score  : 0.9887
precision score : 0.9939
recall score    : 0.9897
f1 score        : 0.9918
EPOCH: 0101
Train loss: 0.0002	Train

EPOCH: 0129
Train loss: 0.0003	Train acc : 0.6087	Valid loss: 0.0002	Valid acc : 0.6095
EPOCH: 0129 prediction results 
confusion matrix
[[ 733   12]
 [  15 1638]]
accuracy score  : 0.9887
precision score : 0.9927
recall score    : 0.9909
f1 score        : 0.9918
EPOCH: 0130
Train loss: 0.0002	Train acc : 0.6097	Valid loss: 0.0003	Valid acc : 0.6104
EPOCH: 0130 prediction results 
confusion matrix
[[ 735   10]
 [  17 1636]]
accuracy score  : 0.9887
precision score : 0.9939
recall score    : 0.9897
f1 score        : 0.9918
EPOCH: 0131
Train loss: 0.0002	Train acc : 0.6097	Valid loss: 0.0002	Valid acc : 0.6108
EPOCH: 0131 prediction results 
confusion matrix
[[ 735   10]
 [  17 1636]]
accuracy score  : 0.9887
precision score : 0.9939
recall score    : 0.9897
f1 score        : 0.9918
EPOCH: 0132
Train loss: 0.0003	Train acc : 0.6089	Valid loss: 0.0002	Valid acc : 0.6108
EPOCH: 0132 prediction results 
confusion matrix
[[ 735   10]
 [  16 1637]]
accuracy score  : 0.9892
precision score : 0

In [7]:
model.load_state_dict(torch.load('weights/tumor_classification/best_resnet50.pt'))

pred, true = predict(model, test_iterator)
confusion_mat, accuracy, precision, recall, f1 = compute_test_metrics(true, pred)
        
print_test_log(epoch_num, accuracy, precision, recall, f1)

EPOCH: 0133 prediction results 
confusion matrix
[[ 736    9]
 [  17 1636]]
accuracy score  : 0.9892
precision score : 0.9945
recall score    : 0.9897
f1 score        : 0.9921
